<a href="https://colab.research.google.com/github/Shambu4599/TFhousing/blob/master/IMDB_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
!pip install tf_nightly
import tensorflow as tf
!pip install numpy==1.16.1
from __future__ import absolute_import,division,print_function
from tensorflow import keras
import numpy as np

imdb = keras.datasets.imdb

(train_data,train_labels),(test_data,test_labels) = imdb.load_data(num_words = 10000)

print ("Training entries: ",len(train_data))
print("labels",len(test_labels))

Training entries:  25000
labels 25000


In [44]:
print(train_data[1])

[1, 194, 1153, 194, 8255, 78, 228, 5, 6, 1463, 4369, 5012, 134, 26, 4, 715, 8, 118, 1634, 14, 394, 20, 13, 119, 954, 189, 102, 5, 207, 110, 3103, 21, 14, 69, 188, 8, 30, 23, 7, 4, 249, 126, 93, 4, 114, 9, 2300, 1523, 5, 647, 4, 116, 9, 35, 8163, 4, 229, 9, 340, 1322, 4, 118, 9, 4, 130, 4901, 19, 4, 1002, 5, 89, 29, 952, 46, 37, 4, 455, 9, 45, 43, 38, 1543, 1905, 398, 4, 1649, 26, 6853, 5, 163, 11, 3215, 2, 4, 1153, 9, 194, 775, 7, 8255, 2, 349, 2637, 148, 605, 2, 8003, 15, 123, 125, 68, 2, 6853, 15, 349, 165, 4362, 98, 5, 4, 228, 9, 43, 2, 1157, 15, 299, 120, 5, 120, 174, 11, 220, 175, 136, 50, 9, 4373, 228, 8255, 5, 2, 656, 245, 2350, 5, 4, 9837, 131, 152, 491, 18, 2, 32, 7464, 1212, 14, 9, 6, 371, 78, 22, 625, 64, 1382, 9, 8, 168, 145, 23, 4, 1690, 15, 16, 4, 1355, 5, 28, 6, 52, 154, 462, 33, 89, 78, 285, 16, 145, 95]


In [0]:
word_index = imdb.get_word_index()

word_index = {k:(v+3) for k,v in word_index.items()} 
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2  # unknown
word_index["<UNUSED>"] = 3

reverse_word_index = dict([value,key] for [key,value] in word_index.items())

def decode_review(text):
  return ' '.join([reverse_word_index.get(i,'?') for i in text])


In [46]:
decode_review(train_data[12])

"<START> i love cheesy horror flicks i don't care if the acting is sub par or whether the monsters look corny i liked this movie except for the <UNK> feeling all the way from the beginning of the film to the very end look i don't need a 10 page <UNK> or a sign with big letters explaining a plot to me but dark floors takes the what is this movie about thing to a whole new annoying level what is this movie about br br this isn't exceptionally scary or thrilling but if you have an hour and a half to kill and or you want to end up feeling frustrated and confused rent this winner"

In [0]:
train_data = keras.preprocessing.sequence.pad_sequences(train_data,value = word_index["<PAD>"],padding = 'post',maxlen = 256)
test_data = keras.preprocessing.sequence.pad_sequences(train_data,value = word_index["<PAD>"],padding = 'post',maxlen = 256)

In [17]:
print(len(train_data[1]),len(test_data[2]))

256 256


In [48]:
vocab_size = 10000

model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size,16))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(16,activation = tf.nn.relu))
model.add(keras.layers.Dense(1,activation = tf.nn.sigmoid))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, None, 16)          160000    
_________________________________________________________________
global_average_pooling1d_2 ( (None, 16)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 17        
Total params: 160,289
Trainable params: 160,289
Non-trainable params: 0
_________________________________________________________________


In [49]:
model.compile(optimizer ='adam',loss = 'binary_crossentropy',metrics = ['acc'])

x_val = train_data[:10000]
partial_x_train = train_data[10000:]

y_val = train_labels[:10000]
partial_y_train = train_labels[10000:]

history = model.fit(partial_x_train,partial_y_train,epochs = 40, batch_size = 512,validation_data = (x_val,y_val),verbose =1)
results = model.evaluate(test_data,test_labels)
print(results)

Train on 15000 samples, validate on 10000 samples
Epoch 1/40
15000/15000 [==============================] - 1s 73us/sample - loss: 0.6925 - acc: 0.5681 - val_loss: 0.6914 - val_acc: 0.6421
Epoch 2/40
15000/15000 [==============================] - 1s 48us/sample - loss: 0.6887 - acc: 0.6865 - val_loss: 0.6850 - val_acc: 0.7342
Epoch 3/40
15000/15000 [==============================] - 1s 48us/sample - loss: 0.6786 - acc: 0.7476 - val_loss: 0.6725 - val_acc: 0.7382
Epoch 4/40
15000/15000 [==============================] - 1s 49us/sample - loss: 0.6611 - acc: 0.7609 - val_loss: 0.6519 - val_acc: 0.7399
Epoch 5/40
15000/15000 [==============================] - 1s 48us/sample - loss: 0.6338 - acc: 0.7808 - val_loss: 0.6225 - val_acc: 0.7677
Epoch 6/40
15000/15000 [==============================] - 1s 49us/sample - loss: 0.5972 - acc: 0.8005 - val_loss: 0.5858 - val_acc: 0.7941
Epoch 7/40
15000/15000 [==============================] - 1s 48us/sample - loss: 0.5537 - acc: 0.8171 - val_loss: 0.

In [50]:
history_dict = history.history
history_dict.keys()


dict_keys(['loss', 'acc', 'val_loss', 'val_acc'])

In [51]:
import matplotlib.pyplot as plt

acc = history_dict['acc']
loss = history_dict['loss']
val_loss = history_dict['val_loss']
val_acc = history_dict['val_acc']
epochs = range(1,len(acc) +1)


# "bo" is for "blue dot"
plt.plot(epochs, loss, 'bo', label='Training loss')
# b is for "solid blue line"
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [52]:
plt.clf()   # clear figure

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()